In [1]:
15*60

900

### create Lambda using py SDK

In [2]:
import boto3

client = boto3.client('lambda', region_name='eu-west-2')
function_name='model_training'
source_arn = 'arn:aws:s3:::cmg-sagemaker-compliance-cases-data/arrears-compliance-processed-data/'
bucket = 'cmg-sagemaker-compliance-cases-data'

response = client.create_function(
    FunctionName=function_name,
    Runtime='python3.8',
    Role='arn:aws:iam::798861612849:role/service-role/AmazonSageMaker-ExecutionRole-20221020T100790',
    Handler='lambda_function.lambda_handler',
    Code={
        'S3Bucket': 'cmg-sagemaker-compliance-case-shared-notebooks',
        'S3Key': 'lambda_function.zip'
    },
    Description='This lambda function invokes a python script that starts a sagemaker xgboost container instance for training a model',
    Timeout=900,
    MemorySize=128,
    Publish=True,
)

response = client.add_permission(
    FunctionName=function_name,
    StatementId='s3-trigger',
    Action='lambda:InvokeFunction',
    Principal='s3.amazonaws.com',
    SourceArn=source_arn,
)

response = client.add_permission(
    FunctionName=function_name,
    StatementId='s3-trigger-2',
    Action='lambda:InvokeFunction',
    Principal='s3.amazonaws.com',
    SourceArn= f'{source_arn}/*',
)

s3_client = boto3.client('s3', region_name='eu-west-2')

response = s3_client.put_bucket_notification_configuration(
    Bucket= bucket,
    NotificationConfiguration={
        'LambdaFunctionConfigurations': [
            {
                'LambdaFunctionArn': f'arn:aws:lambda:eu-west-2:798861612849:function:{function_name}',
                'Events': [
                    's3:ObjectCreated:*'
                ],
                'Filter': {
                    'Key': {
                        'FilterRules': [
                            {
                                'Name': 'prefix',
                                'Value': 'arrears-compliance-processed-data/30-04-2023-train-test/'
                            }
                        ]
                    }
                }
            }
        ]
    }
)


ClientError: An error occurred (AccessDeniedException) when calling the CreateFunction operation: User: arn:aws:sts::798861612849:assumed-role/AmazonSageMaker-ExecutionRole-20221020T100790/SageMaker is not authorized to perform: lambda:CreateFunction on resource: arn:aws:lambda:eu-west-2:798861612849:function:model_training because no identity-based policy allows the lambda:CreateFunction action

### create Lambda using CLI

In [ ]:
aws lambda create-function \
    --function-name model_training \
    --zip-file fileb://lambda_function.zip \
    --handler lambda_function.lambda_handler \
    --runtime python3.8 \
    --role arn:aws:iam::798861612849:role/service-role/AmazonSageMaker-ExecutionRole-20221020T100790 \
    --region eu-west-2
                        



In [ ]:
aws lambda create-function \
    --function-name model_training \
    --code S3Bucket='cmg-sagemaker-compliance-case-shared-notebooks',S3Key='lambda_function.zip' \
    --handler lambda_function.lambda_handler \
    --runtime python3.8 \
    --role arn:aws:iam::798861612849:role/service-role/AmazonSageMaker-ExecutionRole-20221020T100790 \
    --region eu-west-2


### add permissions 

In [ ]:
aws lambda add-permission \
    --function-name model_training \
    --statement-id s3-trigger \
    --action lambda:InvokeFunction \
    --principal s3.amazonaws.com \
    --source-arn arn:aws:s3:::cmg-sagemaker-compliance-cases-data/arrears-compliance-processed-data/ \
    --region eu-west-2

aws lambda add-permission \
    --function-name model_training \
    --statement-id s3-trigger-2 \
    --action lambda:InvokeFunction \
    --principal s3.amazonaws.com \
    --source-arn arn:aws:s3:::cmg-sagemaker-compliance-cases-data/arrears-compliance-processed-data/* \
    --region eu-west-2

aws s3api put-bucket-notification-configuration \
    --bucket cmg-sagemaker-compliance-cases-data \
    --notification-configuration file://notification.json \
    --region eu-west-2


### create AIM permissions policy to create Lambda functions.

In [3]:
import boto3
import json

iam = boto3.client('iam')

policy_document = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "lambda:CreateFunction",
            "Resource": "*"
        }
    ]
})

response = iam.create_policy(
    PolicyName='my-lambda-create-function-policy',
    PolicyDocument=policy_document
)

policy_arn = response['Policy']['Arn']

response = iam.attach_role_policy(
    RoleName='AmazonSageMaker-ExecutionRole-20221020T100790',
    PolicyArn=policy_arn
)


ClientError: An error occurred (AccessDenied) when calling the CreatePolicy operation: User: arn:aws:sts::798861612849:assumed-role/AmazonSageMaker-ExecutionRole-20221020T100790/SageMaker is not authorized to perform: iam:CreatePolicy on resource: policy my-lambda-create-function-policy because no identity-based policy allows the iam:CreatePolicy action